https://kedro.readthedocs.io/en/stable/12_faq/01_faq.html#what-is-data-engineering-convention

In [1]:
import numpy as np
import pandas as pd

In [2]:
%reload_kedro

2021-03-28 20:23:46,815 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-03-28 20:23:46,921 - root - INFO - ** Kedro project xflats
2021-03-28 20:23:46,923 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-03-28 20:23:46,938 - root - INFO - Registered line magic `run_viz`


In [3]:
df = catalog.load('preprocessed_flats')

2021-03-28 20:23:46,957 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_flats` (ParquetDataSet)...


## Features - prices in neighbourhood

In [4]:
df.columns

Index(['_id', 'tracking_id', 'price', 'name', 'location', 'flat_size', 'rooms',
       'floor', 'producer_name', 'price_m2', 'market', 'building_type',
       'description', 'additional_info', 'number_of_floors',
       'building_material', 'year_of_building', 'property_form',
       'download_date', 'download_date_utc', 'offeror', 'date_created',
       'date_modified', 'GC_latitude', 'GC_longitude', 'GC_boundingbox',
       'GC_addr_road', 'GC_addr_neighbourhood', 'GC_addr_suburb',
       'GC_addr_city', 'GC_addr_state', 'GC_addr_postcode', 'GC_addr_country',
       'GC_addr_country_code', 'prediction', 'date_offer'],
      dtype='object')

In [5]:
df.GC_addr_suburb.value_counts()


Wola              22092
Mokotów           22047
Śródmieście       17416
Białołęka         17020
Praga-Południe    13841
Ursynów            9432
Bemowo             8923
Bielany            7429
Ursus              7202
Targówek           6229
Ochota             5966
Wilanów            5207
Włochy             4825
Praga-Północ       4417
Warszawa           4203
Żoliborz           3728
Wawer              3220
Sielce             1340
Rembertów          1113
Wesoła              680
Natolin             286
Imielin              11
Kabaty                8
Wolica                1
Name: GC_addr_suburb, dtype: int64

In [6]:
_districts = list(pd.DataFrame(df.GC_addr_suburb.value_counts()).query("GC_addr_suburb > 1000").index)
_districts = [i for i in _districts if i not in ['Warszawa']]

df.GC_addr_suburb = [i if i in _districts else np.nan for i in df.GC_addr_suburb]
df['date_offer_d'] = df.date_offer.dt.floor("D")


2021-03-28 20:23:50,048 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [7]:
date_max = df['date_offer_d'].max()

In [8]:
date_max

Timestamp('2021-03-27 00:00:00')

In [9]:
for sub in df.GC_addr_suburb.unique():
    for mar in df.market.unique():
        _date = df.query(f"GC_addr_suburb=='{sub}' and market=='{mar}'")['date_offer_d'].max()
        if _date != date_max:
            print(sub, mar, _date)

Bemowo pierwotny 2021-03-25 00:00:00
Bemowo wtorny 2021-03-26 00:00:00
Ursynów pierwotny 2021-03-26 00:00:00
Ursynów wtorny 2021-03-26 00:00:00
Wola pierwotny 2021-03-26 00:00:00
Wawer pierwotny 2021-03-26 00:00:00
Wawer wtorny 2021-03-26 00:00:00
Ursus pierwotny 2021-03-26 00:00:00
Ursus wtorny 2021-03-26 00:00:00
Śródmieście pierwotny 2021-03-26 00:00:00
nan pierwotny NaT
nan wtorny NaT
Praga-Północ pierwotny 2021-03-26 00:00:00
Praga-Północ wtorny 2021-03-26 00:00:00
Bielany pierwotny 2021-03-26 00:00:00
Bielany wtorny 2021-03-26 00:00:00
Praga-Południe pierwotny 2021-03-26 00:00:00
Praga-Południe wtorny 2021-03-26 00:00:00
Targówek pierwotny 2021-03-24 00:00:00
Targówek wtorny 2021-03-26 00:00:00
Ochota pierwotny 2021-03-22 00:00:00
Ochota wtorny 2021-03-26 00:00:00
Wilanów pierwotny 2021-03-26 00:00:00
Wilanów wtorny 2021-03-26 00:00:00
Mokotów pierwotny 2021-03-26 00:00:00
Mokotów wtorny 2021-03-26 00:00:00
Włochy pierwotny 2021-03-26 00:00:00
Włochy wtorny 2021-03-26 00:00:00
Żo

In [10]:
df.set_index('date_offer_d', inplace=True)

#period_length 
tmp01 = df[['GC_addr_suburb','market','price']].groupby(['GC_addr_suburb','market']).resample('W')

_kind = ['median','mean']
_weeks = [1, 2, 3, 4, 8, 12]
results = list()

_start = None

for _k in _kind:
    for _w in _weeks:
        tmp01_agr = tmp01.aggregate(_k)
        # calculation not including day 0 - ex. for 2021-03-30, 7 days is 2021-03-29 -> 2021-03-23
        tmp01_agr_last_x_weeks= tmp01_agr.rolling(_w, min_periods=0).aggregate(_k).shift(1).round(0)
        tmp01_agr_last_x_weeks = tmp01_agr_last_x_weeks.reset_index()
        _col_name = f'price_{_k}_{str(_w).zfill(2)}w'
        tmp01_agr_last_x_weeks.columns = ['GC_addr_suburb','market','date_offer_d',_col_name]
        if _start:
            df_aggr[_col_name] = tmp01_agr_last_x_weeks[_col_name]
        else:
            _start = 1
            df_aggr = tmp01_agr_last_x_weeks.copy()


In [11]:
df_aggr

,GC_addr_suburb,market,date_offer_d,price_median_01w,price_median_02w,price_median_03w,price_median_04w,price_median_08w,price_median_12w,price_mean_01w,price_mean_02w,price_mean_03w,price_mean_04w,price_mean_08w,price_mean_12w
0,Bemowo,pierwotny,2020-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bemowo,pierwotny,2020-03-22,610500.0,610500.0,610500.0,610500.0,610500.0,610500.0,705270.0,705270.0,705270.0,705270.0,705270.0,705270.0
2,Bemowo,pierwotny,2020-03-29,1046257.0,828378.0,828378.0,828378.0,828378.0,828378.0,922631.0,813951.0,813951.0,813951.0,813951.0,813951.0
3,Bemowo,pierwotny,2020-04-05,542640.0,794448.0,610500.0,610500.0,610500.0,610500.0,663662.0,793147.0,763854.0,763854.0,763854.0,763854.0
4,Bemowo,pierwotny,2020-04-12,580000.0,561320.0,580000.0,595250.0,595250.0,595250.0,575461.0,619561.0,720585.0,716756.0,716756.0,716756.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,Żoliborz,wtorny,2021-02-28,665000.0,657000.0,649000.0,632000.0,595000.0,592500.0,713783.0,710513.0,673599.0,673737.0,668146.0,665325.0
1934,Żoliborz,wtorny,2021-03-07,572500.0,618750.0,649000.0,610750.0,594000.0,592500.0,663485.0,688634.0,694837.0,671070.0,669793.0,670124.0
1935,Żoliborz,wtorny,2021-03-14,610000.0,591250.0,610000.0,629500.0,591500.0,610000.0,672270.0,667878.0,683179.0,689195.0,661215.0,668771.0
1936,Żoliborz,wtorny,2021-03-21,465000.0,537500.0,572500.0,591250.0,591500.0,592500.0,512238.0,592254.0,615998.0,640444.0,646403.0,655614.0


In [15]:
df.date_offer.max()

Timestamp('2021-03-27 00:45:10')

In [13]:
df_aggr.groupby(['GC_addr_suburb','market']).max()['date_offer_d']

GC_addr_suburb  market   
Bemowo          pierwotny   2021-03-28
                wtorny      2021-03-28
Białołęka       pierwotny   2021-03-28
                wtorny      2021-03-28
Bielany         pierwotny   2021-03-28
                wtorny      2021-03-28
Mokotów         pierwotny   2021-03-28
                wtorny      2021-03-28
Ochota          pierwotny   2021-03-28
                wtorny      2021-03-28
Praga-Południe  pierwotny   2021-03-28
                wtorny      2021-03-28
Praga-Północ    pierwotny   2021-03-28
                wtorny      2021-03-28
Rembertów       pierwotny   2021-03-28
                wtorny      2021-03-28
Sielce          pierwotny   2020-06-14
                wtorny      2020-06-14
Targówek        pierwotny   2021-03-28
                wtorny      2021-03-28
Ursus           pierwotny   2021-03-28
                wtorny      2021-03-28
Ursynów         pierwotny   2021-03-28
                wtorny      2021-03-28
Wawer           pierwotny   2021-03-28